In [1]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt

from config import Config
import utils
import model as modellib
import visualize
from model import log

%matplotlib inline

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = "/Users/wenyuan/Documents/MII/Mask_RCNN_On_Pathology/Mask_RCNN/mask_rcnn_coco.h5"

Using TensorFlow backend.


In [2]:
import prostate
dataset_dir = "/Users/wenyuan/Documents/MII/Mask_RCNN_On_Pathology/Data_Pre_Processing/cedars-224"
held_out_set = 4
mean_pixel = prostate.Mean_pixel(dataset_dir, held_out_set)
class TrainConfig(prostate.ProstateConfig):
            MEAN_PIXEL = np.array(mean_pixel)
config = TrainConfig()
config.display()


Configurations:
BACKBONE_SHAPES                [[128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]
 [  8   8]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     4
BBOX_STD_DEV                   [ 0.1  0.1  0.2  0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.99
GPU_COUNT                      1
IMAGES_PER_GPU                 4
IMAGE_MAX_DIM                  512
IMAGE_MIN_DIM                  512
IMAGE_PADDING                  True
IMAGE_SHAPE                    [512 512   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [ 193.97800579  120.89113632  183.79060979]
MINI_MASK_SHAPE                (56, 56)
NAME                           prostate
NUM_CLASSES                    4
POOL_SIZE                      7
POST_NMS_ROIS_INFERENCE        1000

In [3]:
subset_1 = [0, 3]
dataset_train = prostate.ProstateDataset()
dataset_train.load_prostate(dataset_dir, subset_1, mode = 16)
dataset_train.prepare() # prepare the dataset class for use

In [6]:
# Create model in training mode
model = modellib.ResNet_Classifier(mode="training", config=config,
                          model_dir=MODEL_DIR)

In [7]:
exclude = []
if config.USE_TUMORCLASS:
    tumor_head = ["tumor_class_conv1", "tumor_class_bn1", "tumor_class_conv2",
                 "tumor_class_conv2", "tumor_class_bn2", "tumor_class_logits", "tumor_class"]
    exclude = exclude + tumor_head

model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=exclude)

In [8]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.
model.train(dataset_train, dataset_train, 
            learning_rate=config.LEARNING_RATE, 
            epochs=5, 
            layers='heads')

TypeError: train() got an unexpected keyword argument 'learning_rate'